<a href="https://colab.research.google.com/github/KareemSarhan/Lane-detection/blob/master/Lanes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import google.colab.patches
import numpy as np
import matplotlib.pyplot as plt

def makecoord(img,lines):
    slope , intercept = lines
    y1=img.shape[0]
    y2=int(y1*3/5)
    x1=int((y1-intercept)/slope)
    x2=int((y2-intercept)/slope)

    return np.array([x1,y1,x2,y2])
def argslope(img,lines):
    leftfit=[]
    rightfit=[]
    for line in lines:
        x1,y1,x2,y2 = line.reshape(4)
        param = np.polyfit((x1,x2),(y1,y2),1)
        #print(param)
        slope = param[0]
        intercept=param[1]
        if slope <0:
            leftfit.append((slope,intercept))
        else:    
            rightfit.append((slope,intercept))
    leftfit_avg=np.average(leftfit,axis=0)
    rightfit_avg=np.average(rightfit,axis=0)
    leftline=makecoord(img,leftfit_avg)
    rightline=makecoord(img,rightfit_avg)
    return np.array([leftline,rightline])
def canny(img):
    laneimg =np.copy(img)
    gray=cv2.cvtColor(laneimg,cv2.COLOR_RGB2GRAY)
    #blur =cv2.GaussianBlur(gray,(5,5),0)
    canny = cv2.Canny(gray,50,150)
    return canny
def roi(img):
    height = img.shape[0]
    tri = np.array([[(200,height),(1100,height),(600,250)]])
    mask = np.zeros_like(img)
    cv2.fillPoly(mask,tri,255)
    mskedimg = cv2.bitwise_and(img,mask)
    return mskedimg
def DisplayLines(img,lines):
    line_image=np.zeros_like(img)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2=line.reshape(4)
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    return line_image
def submain1(img):
    tempimg=np.copy(img)
    cannyimg = canny(img)
    roiimg = roi(cannyimg)
    linesimg = cv2.HoughLinesP(roiimg,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
    averaged_lines=argslope(tempimg,linesimg)
    DisplayLinesimg=DisplayLines(img,averaged_lines)
    combo_img=cv2.addWeighted(img,0.8,DisplayLinesimg,1,1)
    cv2_imshow(combo_img)

cap = cv2.VideoCapture("/test2.mp4")
while(cap.isOpened()):
    _, frame = cap.read()
    submain1(frame)



